In [2]:
# ! pip install backoff

In [4]:
# ! pip install  --upgrade flash_attn

In [6]:
! git lfs install
! git clone https://huggingface.co/microsoft/Phi-4-multimodal-instruct

Error: Failed to call git rev-parse --git-dir: exit status 128 
Git LFS initialized.
fatal: destination path 'Phi-4-multimodal-instruct' already exists and is not an empty directory.


⚡️ Tip	Check organization access: https://github.com/settings/connections/applications/c7457225b242a94d60c6



In [ ]:
import requests
import torch
from PIL import Image
import soundfile
from transformers import AutoModelForCausalLM, AutoProcessor, GenerationConfig,pipeline,AutoTokenizer

In [8]:
model_path = './Phi-4-multimodal-instruct'

In [9]:
kwargs = {}
kwargs['torch_dtype'] = torch.float16

In [10]:
processor = AutoProcessor.from_pretrained(model_path, trust_remote_code=True)

/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/transformers/models/auto/image_processing_auto.py:590: FutureWarning: The image_processor_class argument is deprecated and will be removed in v4.42. Please use `slow_image_processor_class`, or `fast_image_processor_class` instead
  warnings.warn(
Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.48, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


In [ ]:
model = AutoModelForCausalLM.from_pretrained(
    model_path,
    trust_remote_code=True,
    torch_dtype='auto',
    _attn_implementation='flash_attention_2',
).cuda()

In [13]:
generation_config = GenerationConfig.from_pretrained(model_path, 'generation_config.json')

In [14]:
system_prompt = '<|system|>'
user_prompt = '<|user|>'
assistant_prompt = '<|assistant|>'
prompt_suffix = '<|end|>'

In [15]:
from PIL import Image

In [16]:
prompt = f"""{system_prompt}You are a technology journalist who writes professional content based on audio and picture content{prompt_suffix}{user_prompt}
Reorganize the content provided by audio <|audio_1|> and image <|image_1|> from a professional perspective and write a press release based on the current development of AI ,output is markdown format and including title and content {prompt_suffix}{assistant_prompt}"""

In [17]:
image_path = "blackhole.jpeg"
image = Image.open(image_path)

In [18]:
audio = soundfile.read('./blackhole.mp3')

In [19]:
inputs = processor(text=prompt, images=[image], audios=[audio], return_tensors='pt').to('cuda:0')

In [31]:
model.__class__

transformers_modules.Phi-4-multimodal-instruct.modeling_phi4mm.Phi4MMForCausalLM

In [25]:
generate_ids = model.generate(
    **inputs,
    max_new_tokens=1500,
    generation_config=generation_config,
)

/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/torch/utils/checkpoint.py:90: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(


In [27]:
generate_ids = generate_ids[:, inputs['input_ids'].shape[1] :]

In [28]:
response = processor.batch_decode(
    generate_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False
)[0]

In [29]:
print(response)

# Press Release

## Title: AI Breakthrough: Unveiling the Mysteries of Black Holes

### Date: [Insert Date]

### Location: [Insert Location]

In a groundbreaking development in the field of astrophysics, a team of AI researchers has made a significant breakthrough in understanding the enigmatic nature of black holes. The latest AI model, designed to simulate and analyze the complex gravitational phenomena associated with these cosmic entities, has provided unprecedented insights into their structure and behavior.

The AI, which has been trained on a vast array of astronomical data, has successfully predicted the behavior of black holes with remarkable accuracy. This achievement marks a significant step forward in our quest to comprehend the universe's most mysterious objects.

The AI's ability to process and interpret data at an extraordinary scale has allowed scientists to visualize the event horizon of black holes, the point of no return where not even light can escape. The model has